In [1]:
import mediapy as media
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [3]:
#constants 
assets_path = "./assets"
video1_file_name = 'video1_category1.mp4'
video2_file_name = 'video2_category1.mp4'
video3_file_name = 'video3_category2.mp4'
video4_file_name = 'video4_category2.mp4'

def get_path_to_video(video_file_name):
    return os.path.join(assets_path, video_file_name)

def get_video(path):
    return  media.read_video(path)

def get_grayscale_image(video):

'./assets/video1_category1.mp4'